# Lab 1

In [ ]:
# Code Block 1
cw.plot(cap_pass_trace("h\n")) * cw.plot(cap_pass_trace("0\n"))

In [ ]:
# Code Block 2
from tqdm.notebook import tqdm
fig = cw.plot()
for c in tqdm('abcdefghijklmnopqrstuvwxyz0123456789'):
    trace = cap_pass_trace(c + "\n")
    fig *= cw.plot(trace[0:1000])
    
display(fig)

In [ ]:
# Code Block 3
ref_trace = cap_pass_trace("h0p\x00\n")[0:1000]
plot = cw.plot()
for c in 'abcdefghijklmnopqrstuvwxyz0123456789': 
    trace = cap_pass_trace('h0p' + c + "\n")[0:1000]
    plot *= cw.plot(trace - ref_trace)
display(plot)

In [ ]:
# Code Block 4
ref_trace = cap_pass_trace( "h0p\x00\n")


for c in 'abcdefghijklmnopqrstuvwxyz0123456789': 
    trace = cap_pass_trace("h0p" + c + "\n")
    diff = np.sum(np.abs(trace - ref_trace))
    
    print("{:1} diff = {:2}".format(c, diff))

In [ ]:
# Code Block 5
guessed_pw = ""

for _ in range(0, 5):  
    biggest_diff = 0
    biggest_char = '\x00'
    ref_trace = cap_pass_trace(guessed_pw + "\x00\n")
    
    for c in 'abcdefghijklmnopqrstuvwxyz0123456789': 
        trace = cap_pass_trace(guessed_pw + c + "\n")
        diff = np.sum(np.abs(trace - ref_trace))

        if diff > biggest_diff:
            biggest_diff = diff
            biggest_char = c
            
    guessed_pw += biggest_char
    print(guessed_pw)

# Lab 2

Lab 2 has no code blocks for the user to fill in

# Lab 3

In [ ]:
# Code Block 1
target.simpleserial_write('k', key)

In [ ]:
# Code Block 2
scope.arm()

target.simpleserial_write('p', text)

ret = scope.capture()
if ret:
    print("Target timed out!")
    continue

response = target.simpleserial_read('r', 16)

trace_array.append(scope.get_last_trace())
textin_array.append(text)

In [ ]:
# Code Block 3
guess_array = np.array([
    [trace_guesser(key_guess, text) for text in texts],
    list(traces)
])
return abs(np.corrcoef(guess_array)[1][0])

In [ ]:
# Code Block 4
return key ^ text

In [ ]:
# Code Block 5
return sbox[state]

In [ ]:
# Code Block 6
def get_correlation(key_guess, texts, traces):
    guess_array = np.array([
        [trace_guesser(key_guess, text) for text in texts],
        list(traces)
    ])
    return abs(np.corrcoef(guess_array)[1][0])

# Now we'll use get_correlation() to break the traces 

best_guess = 0
best_corr = 0
for key_guess in trange(0, 256):
    cur_corr = get_correlation(key_guess, texts, traces)
    if abs(cur_corr) > best_corr:
        best_guess = key_guess
        best_corr = abs(cur_corr)
        print("New best guess found: {:02X} (corr={})".format(best_guess, best_corr))
        


# Lab 4

In [ ]:
# Code block for plotting the guess

import numpy as np
from tqdm.notebook import trange, tqdm

plot = cw.plot()

for i in range(20, 23):
    reset_target()
    target.flush()
    
    response, responsehex = tx_rx(b"\x86", 1, 1)
    if responsehex[0] != 0x86:
        raise IOError("Sync Error")

    response, responsehex = tx_rx(b"\x60", 1, 1)

    if responsehex[0] != 0x60:
        raise IOError("Unexpected ACK = 0x%x"%responsehex[0])
        
    scope.arm()
    target.write(chr(i))
    scope.capture()
    trace = scope.get_last_trace()
      
    plot *= cw.plot(trace, alpha=0.25)
    
display(plot)

In [ ]:
# Code block for plotting the guess with resyncronized traces

import numpy as np
from tqdm.notebook import trange, tqdm

go = True

i = 0

diffs = []
traces = []

plot = cw.plot()

for i in range(20, 24):
    reset_target()
    target.flush()
    response, responsehex = tx_rx(b"\x86", 1, 1)
    if responsehex[0] != 0x86:
        raise IOError("Sync Error")

    response, responsehex = tx_rx(b"\x60", 1, 1)

    if responsehex[0] != 0x60:
        raise IOError("Unexpected ACK = 0x%x"%responsehex[0])
        
    scope.arm()
    target.write(chr(i))
    scope.capture()
    trace = scope.get_last_trace()
    
    #NOTE - you'll need to adjust the -300 number currently to target the center of the actual work
    
    #try:
    trace = resync_sad(trace, trace1, (start,end))[start-300:end-300]
    #except ValueError:
    #    continue
    
    diff_trace = trace - trace1[start:end]
    traces.append(trace)
    
    diff = np.sum(abs(diff_trace)[80:180])
    diffs.append(diff)
    print("%x %f"%(i, diff))
    
    plot *= cw.plot(trace)
    
display(plot)